In [1]:
import itertools
from heapq import heappush, heappop

In [2]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

class Vertex:
    def __init__(self, value):
        self.value = value


class Edge:
    def __init__(self, distance, vertex):
        self.distance = distance
        self.vertex = vertex


def dijkstra(graph, start, end):
    previous = {v: None for v in graph.adjacency_list.keys()}
    visited = {v: False for v in graph.adjacency_list.keys()}
    distances = {v: float("inf") for v in graph.adjacency_list.keys()}
    distances[start] = 0
    queue = PriorityQueue()
    queue.add_task(0, start)
    path = []
    while queue:
        removed_distance, removed = queue.pop_task()
        visited[removed] = True
        if removed is end:
            while previous[removed]:
                path.append(removed.value)
                removed = previous[removed]
            path.append(start.value)
            print(f"shortest distance to {end.value}: ", distances[end])
            print(f"path to {end.value}: ", path[::-1])
            return

        for edge in graph.adjacency_list[removed]:
            if visited[edge.vertex]:
                continue
            new_distance = removed_distance + edge.distance
            if new_distance < distances[edge.vertex]:
                distances[edge.vertex] = new_distance
                previous[edge.vertex] = removed
                queue.add_task(new_distance, edge.vertex)
    return

class PriorityQueue:
    def __init__(self):
        self.pq = []
        self.entry_finder = {}
        self.counter = itertools.count()

    def __len__(self):
        return len(self.pq)

    def add_task(self, priority, task):
        if task in self.entry_finder:
            self.update_priority(priority, task)
            return self
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def update_priority(self, priority, task):
        entry = self.entry_finder[task]
        count = next(self.counter)
        entry[0], entry[1] = priority, count

    def pop_task(self):
        while self.pq:
            priority, count, task = heappop(self.pq)
            del self.entry_finder[task]
            return priority, task


In [ ]:
# vertices = [ Vertex('A'), Vertex('B'), Vertex('C'), Vertex('D'), Vertex('E')]

# A, B, C, D, E = vertices

# adjacency_list = {
#   A: [Edge(4, B), Edge(2, C)],
#   B: [Edge(4, A), Edge(3, C), Edge(2, D), Edge(3, E)],
#   C: [Edge(2, A), Edge(3, B), Edge(4, D), Edge(5, E)],
#   D: [Edge(2, B), Edge(4, C), Edge(1, E)],
#   E: [Edge(3, B), Edge(5, C), Edge(1, D)],
# }

# my_graph = Graph(adjacency_list)


# dijkstra(my_graph, start=A, end=D)

shortest distance to D:  6
path to D:  ['A', 'C', 'D']


In [ ]:
# adjacency_list

{<__main__.Vertex at 0x1baf59f2950>: [<__main__.Edge at 0x1baf579fdd0>,
 <__main__.Vertex at 0x1baf59b0440>: [<__main__.Edge at 0x1baf5db0100>,
 <__main__.Vertex at 0x1baf59b0750>: [<__main__.Edge at 0x1baf59f0ad0>,
 <__main__.Vertex at 0x1baf59c70b0>: [<__main__.Edge at 0x1baf5d959d0>,
 <__main__.Vertex at 0x1baf5d95d30>: [<__main__.Edge at 0x1baf5d95f10>,
  <__main__.Edge at 0x1baf5d96030>]}

In [4]:
import psycopg2

koneksi = psycopg2.connect(
    host="localhost",
    database="route_planner",
    user="postgres",
    password=""
)

cur = koneksi.cursor()

cur.execute("""
SELECT 
    t1a.bus_stop_name AS stop_1,
    t1b.bus_stop_name AS stop_2,
    rutes.distance
FROM 
    rutes
    JOIN bus_stops t1a ON rutes.stop_1::integer = t1a.id
    JOIN bus_stops t1b ON rutes.stop_2::integer = t1b.id;
""")

rows = cur.fetchall()

for row in rows:
    print(row)

colnames = [desc[0] for desc in cur.description]
print("Columns:", colnames)

cur.close()
koneksi.close()


('A', 'B', 4)
('A', 'C', 2)
('B', 'A', 4)
('B', 'C', 3)
('B', 'D', 2)
('B', 'E', 3)
('C', 'A', 2)
('C', 'B', 3)
('C', 'E', 5)
('C', 'D', 4)
('D', 'B', 2)
('D', 'C', 4)
('D', 'E', 1)
('E', 'C', 5)
('E', 'B', 3)
('E', 'D', 1)
Columns: ['stop_1', 'stop_2', 'distance']


In [5]:
import pandas as pd
df = pd.DataFrame(rows, columns=colnames)
print(df)

   stop_1 stop_2  distance
0       A      B         4
1       A      C         2
2       B      A         4
3       B      C         3
4       B      D         2
5       B      E         3
6       C      A         2
7       C      B         3
8       C      E         5
9       C      D         4
10      D      B         2
11      D      C         4
12      D      E         1
13      E      C         5
14      E      B         3
15      E      D         1


In [6]:
temp = []
for stop_1 in df['stop_1'].unique():
    temp2 = []
    for val in df.where(df['stop_1'] == stop_1).dropna().iloc[:,1:].values:
        temp2.append(Edge(int(val[1]), Vertex(val[0])))
    temp.append(temp2)
temp
        

[[<__main__.Edge at 0x1baf55fb0e0>, <__main__.Edge at 0x1baf567ae90>],

In [7]:
stops = []
adjacency_lists = {}
for i, stop_1 in enumerate(df['stop_1'].unique()):
    ver = Vertex(stop_1)
    adjacency_lists[ver] = temp[i]
    stops.append(ver)

In [8]:
stops

In [9]:
adjacency_lists

{<__main__.Vertex at 0x1baf565b1d0>: [<__main__.Edge at 0x1baf55fb0e0>,
 <__main__.Vertex at 0x1baf56d59c0>: [<__main__.Edge at 0x1baf567b250>,
 <__main__.Vertex at 0x1baf56d5b20>: [<__main__.Edge at 0x1baf568f020>,
 <__main__.Vertex at 0x1bad55db9d0>: [<__main__.Edge at 0x1baf56ca300>,
 <__main__.Vertex at 0x1baf57298b0>: [<__main__.Edge at 0x1baf570a0b0>,
  <__main__.Edge at 0x1baf565aed0>]}

In [10]:
my_graph = Graph(adjacency_lists)
dijkstra(my_graph, start=Vertex('A'), end=Vertex('D'))

KeyError: <__main__.Vertex object at 0x000001BAD54817F0>